In [4]:
from google.cloud.storage.blob import Blob
from google.cloud import storage
import os
import sys
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import csv

In [5]:
df = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/distances_CompA_V6.csv', delimiter=',', encoding='utf-8', header=0)
print(df.head(20))
print(df.info())

                                    ORGANISATION_NAME  REGISTRATION_NO  \
0                    THE SUN INSURANCE OFFICE LIMITED               11   
1                     COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                                    SIMMONS BROTHERS               32   
3                                  NEW STAR SHOE SHOP              118   
4                           CHARALAMBOS PIERI AND SON               17   
5   ΚΥΠΡΙΑΚΟΝ ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗ...               19   
6   ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗΡΙΟΝ ΛΕΥΚΩ...              502   
7   Ο ΚΥΠΡΙΑΚΟΣ ΣΥΝΔΕΣΜΟΣ ΙΔΙΟΚΤΗΤΩΝ ΑΚΙΝΗΤΩΝ ΛΙΜΙΤΕΔ             1293   
8                CYPRUS-AMERICAN BUSINESS ASSOCIATION            17284   
9                                               CYABA            17285   
10                                      AMCHAM CYPRUS            35841   
11                                   C.J. CACOYIANNIS               28   
12                                    

In [6]:
#Following investigation, Cyprus lies at a latitude of 34˚33˚ - 35˚34˚ North and longitude 32˚16˚ - 34˚37˚ East. 
#Therefore we will exclude addresses that do not fall in the valid range of Cy geocoordinates
#lat1, lon1, lat2, lon2 = 35.2, 34.37, 34.33, 32.16
#Further investigation and given that we were losing 2 service points of the existing service points of CompA,
# as Cyprus cannot be described by a perfect rectangular shape, 
#the below latitude and longitude extremes will be used 
lat1, lon1, lat2, lon2 = 35.2, 34.85, 34.33, 32.16
dist_df = df[(df['lat'] <= lat1) & (df['lat'] >= lat2) & (df['lng'] <= lon1) & (df['lng'] >= lon2)]
dist_df

ORGANISATION_NAME  REGISTRATION_NO  \
0       THE SUN INSURANCE OFFICE LIMITED               11   
1        COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                       SIMMONS BROTHERS               32   
3                     NEW STAR SHOE SHOP              118   
4              CHARALAMBOS PIERI AND SON               17   
...                                  ...              ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED           441880   
218734                      ZI. RA 1 LTD           441941   
218735                 ACLA AUDITORS LTD           441954   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ           442013   
218737                       ILLIDAN LTD           442033   

       ORGANISATION_TYPE_CODE  ORGANISATION_TYPE ORGANISATION_STATUS  \
0                           O  Αλλοδαπή Εταιρεία        Εγγεγραμμένη   
1                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
2                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
3                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
4                           B  Εμπορική Επωνυμία        Εγγεγραμμένη   
...                       ...                ...                 ...   
218733                      C           Εταιρεία        Εγγεγραμμένη   
218734                      C           Εταιρεία        Εγγεγραμμένη   
218735                      C           Εταιρεία        Εγγεγραμμένη   
218736                      C           Εταιρεία        Εγγεγραμμένη   
218737                      C           Εταιρεία        Εγγεγραμμένη   

        address_seq_no                     ADDRESS_L1  \
0               281510     ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12    
1               274852                            NaN   
2               274852                            NaN   
3               274852                            NaN   
4               262535                ARISTIDES ROAD    
...                ...                            ...   
218733          267985  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68    
218734          565102                  ΓΛΑΔΣΤΩΝΟΣ 1    
218735          439357              ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21    
218736          352550                   ΟΔΥΣΣΕΩΣ 10    
218737          195348               ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3    

                            ADDRESS_L3  \
0                            ΛΕΥΚΩΣΙΑ    
1                            ΛΕΥΚΩΣΙΑ    
2                            ΛΕΥΚΩΣΙΑ    
3                            ΛΕΥΚΩΣΙΑ    
4                            ΛΕΥΚΩΣΙΑ    
...                                ...   
218733    ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4152 ΛΕΜΕΣΟΣ    
218734                   6023 ΛΑΡΝΑΚΑ    
218735  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ    
218736        ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ    
218737                   3112 ΛΕΜΕΣΟΣ    

                                             FULL ADDRESS  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ    
1                                               ΛΕΥΚΩΣΙΑ    
2                                               ΛΕΥΚΩΣΙΑ    
3                                               ΛΕΥΚΩΣΙΑ    
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ    
...                                                   ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ    
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ    
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ    
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ    

                                lat_lng  ...     48     49     50     51  \
0              (35.1602214, 33.3604734)  ...  56.38  56.85  61.39  64.01   
1       (35.1855659, 33.38227639999999)  ...  59.79  60.25  64.83  67.45   
2       (35.1855659, 33.38227639999999)  ...  59.79  60.25  64.83  67.45   
3       (35.1855659, 33.38227639999999)  ...  59.79  60.25  64.83  67.45   
4       (35.1855659, 33.38227639999999)  ...  59.79  60.25  64.83  67.45   
...                                 ...  ...  

In [7]:
msr_df = dist_df.select_dtypes(include=['number'])
msr_df = msr_df.drop(['lat_txt','lng_txt','lat','lng','address_seq_no','REGISTRATION_NO'], axis=1)
msr_df['min_dist'] = msr_df.min(axis='columns')
msr_df['min_sp'] = msr_df.idxmin(axis='columns')
msr_df

Valid Seq      1      2      3      4      5      6      7      8  \
0          281510   6.45   1.19   0.67   2.54   3.87   1.92  16.45   3.19   
1          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
2          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
3          274852   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
4          262535   9.05   4.50   2.80   5.98   6.28   4.52  19.01   4.16   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
218733     267985  58.00  61.16  62.86  59.69  60.67  61.62  51.36  62.79   
218734     565102  30.16  34.97  36.14  35.60  38.79  37.40  23.20  32.98   
218735     439357  15.29  14.03  14.91  12.27  11.17  12.92  20.02  16.93   
218736     352550  31.67  36.94  38.43  37.11  40.24  39.24  22.58  35.61   
218737     195348  56.46  59.87  61.61  58.47  59.63  60.48  49.36  61.36   

            9  ...     50     51      52      53      54      55      56  \
0        2.36  ...  61.39  64.01   94.93   94.19   95.48   95.59   95.13   
1        5.08  ...  64.83  67.45   97.95   97.22   98.51   98.46   98.17   
2        5.08  ...  64.83  67.45   97.95   97.22   98.51   98.46   98.17   
3        5.08  ...  64.83  67.45   97.95   97.22   98.51   98.46   98.17   
4        5.08  ...  64.83  67.45   97.95   97.22   98.51   98.46   98.17   
...       ...  ...    ...    ...     ...     ...     ...     ...     ...   
218733  61.13  ...   1.01   3.06   53.95   52.68   53.75   59.70   53.13   
218734  37.53  ...  61.23  65.10  110.18  109.12  110.38  113.35  109.87   
218735  12.36  ...  50.81  52.88   80.53   79.81   81.10   81.12   80.76   
218736  39.23  ...  52.36  56.30  102.85  101.75  102.98  106.49  102.45   
218737  60.01  ...   3.03   6.83   57.77   56.50   57.57   63.49   56.95   

            57  min_dist  min_sp  
0        94.30      0.67       3  
1        97.37      0.57      17  
2        97.37      0.57      17  
3        97.37      0.57      17  
4        97.37      0.57      17  
...        ...       ...     ...  
218733   51.65      1.01      50  
218734  108.65      0.50      30  
218735   79.96      3.93      21  
218736  101.17      3.26      31  
218737   55.47      0.56      39  

[217788 rows x 60 columns]

In [8]:
#Keep only the the interesting columns
serv_pts_df = pd.DataFrame(msr_df, columns=['min_dist', 'min_sp'])
serv_pts_df['min_sp'] = serv_pts_df['min_sp'].astype('int64')
serv_pts_df

min_dist  min_sp
0           0.67       3
1           0.57      17
2           0.57      17
3           0.57      17
4           0.57      17
...          ...     ...
218733      1.01      50
218734      0.50      30
218735      3.93      21
218736      3.26      31
218737      0.56      39

[217788 rows x 2 columns]

In [9]:
serv_pts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217788 entries, 0 to 218737
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   min_dist  217788 non-null  float64
 1   min_sp    217788 non-null  int64  
dtypes: float64(1), int64(1)
memory usage: 5.0 MB


In [10]:
#Concatenate dataframes to get the closest service point per customer
cust_df = pd.DataFrame(dist_df, columns=['ORGANISATION_NAME','address_seq_no', 'FULL ADDRESS', 'lat', 'lng'])
cust_sp_df = pd.concat([cust_df,serv_pts_df],axis=1)
cust_sp_df.rename(columns={'address_seq_no':'AddressID'}, inplace=True)
cust_sp_df.rename(columns={'min_dist':'MinDistTo_CompA_SP'}, inplace=True)
cust_sp_df.rename(columns={'min_sp':'CompA_SP'}, inplace=True)
cust_sp_df

ORGANISATION_NAME  AddressID  \
0       THE SUN INSURANCE OFFICE LIMITED     281510   
1        COMPTOIR D' ESCOMPTE DE NICOSIE     274852   
2                       SIMMONS BROTHERS     274852   
3                     NEW STAR SHOE SHOP     274852   
4              CHARALAMBOS PIERI AND SON     262535   
...                                  ...        ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED     267985   
218734                      ZI. RA 1 LTD     565102   
218735                 ACLA AUDITORS LTD     439357   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ     352550   
218737                       ILLIDAN LTD     195348   

                                             FULL ADDRESS        lat  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ   35.160221   
1                                               ΛΕΥΚΩΣΙΑ   35.185566   
2                                               ΛΕΥΚΩΣΙΑ   35.185566   
3                                               ΛΕΥΚΩΣΙΑ   35.185566   
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ   35.185566   
...                                                   ...        ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...  34.685159   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ   34.918222   
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ   35.114799   
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ   34.854814   
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ   34.678632   

              lng  MinDistTo_CompA_SP  CompA_SP  
0       33.360473                0.67         3  
1       33.382276                0.57        17  
2       33.382276                0.57        17  
3       33.382276                0.57        17  
4       33.382276                0.57        17  
...           ...                 ...       ...  
218733  33.000279                1.01        50  
218734  33.620063                0.50        30  
218735  33.211395                3.93        21  
218736  33.546889                3.26        31  
218737  33.041305                0.56        39  

[217788 rows x 7 columns]

In [11]:
cust_sp_df.to_csv('CompA_MinSP_AllData_V2.csv', encoding='utf-8')

In [2]:
CompA_Loc = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/CompA_Locs_v2.csv', delimiter=';', encoding='utf-8')
CompA_Loc.dropna(inplace=True)
CompA_Loc['AA'] = CompA_Loc['AA'].astype('int64')
CompA_Loc.rename(columns={'AA': 'Service Point'}, inplace=True)
CompA_Loc.head(5)

Service Point                           Full Address   Latitude  Longitude  \
0              1  Ελευθερίου Βενιζέλου 1, Λευκωσία 2235  35.104199  33.379002   
1              2    Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012  35.149497  33.359752   
2              3       Λεωφ. Μακαρίου 39, Λευκωσία 1065  35.165882  33.362994   
3              4     Λεωφ. Στροβόλου 116, Λευκωσία 2042  35.143165  33.341816   
4              5       28ης Οκτωβρίου 8A, Λευκωσία 2414  35.164435  33.318236   

  District  
0  Nicosia  
1  Nicosia  
2  Nicosia  
3  Nicosia  
4  Nicosia

In [11]:
#Some baseline counts regarding existing physical branch network of Company A
customers_per_sp = serv_pts_df.groupby("min_sp").count().reset_index()
customers_per_sp.columns = ['Service Point', 'NoOfCustomers']
customers_per_sp['Service Point'] = customers_per_sp['Service Point'].astype('int64')
customers_per_sp['NoOfCustomers'] = customers_per_sp['NoOfCustomers'].astype('int64')
print(customers_per_sp.head(5))
join_df = pd.merge(CompA_Loc, customers_per_sp, on="Service Point",how="left")
join_df['NoOfCustomers'].fillna(0, inplace=True)
join_df['NoOfCustomers'] = join_df['NoOfCustomers'].astype('int64')
join_df.sort_values('Service Point', ascending=True).head(5)

   Service Point  NoOfCustomers
0              1           1707
1              2           4061
2              3          23857
3              4           5558
4              5           4740


Service Point                           Full Address   Latitude  Longitude  \
0              1  Ελευθερίου Βενιζέλου 1, Λευκωσία 2235  35.104199  33.379002   
1              2    Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012  35.149497  33.359752   
2              3       Λεωφ. Μακαρίου 39, Λευκωσία 1065  35.165882  33.362994   
3              4     Λεωφ. Στροβόλου 116, Λευκωσία 2042  35.143165  33.341816   
4              5       28ης Οκτωβρίου 8A, Λευκωσία 2414  35.164435  33.318236   

  District  NoOfCustomers  
0  Nicosia           1707  
1  Nicosia           4061  
2  Nicosia          23857  
3  Nicosia           5558  
4  Nicosia           4740

In [12]:
#Statistics per service point
stats_per_sp = serv_pts_df.groupby(['min_sp']).aggregate([np.min,np.max,np.mean,np.median]).reset_index()
stats_per_sp.columns = stats_per_sp.columns.map('_'.join)
stats_per_sp.rename(columns={'min_sp_':'Service Point'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_amin':'Min Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_amax':'Max Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_mean':'Mean Distance'}, inplace=True)
stats_per_sp.rename(columns={'min_dist_median':'Median Distance'}, inplace=True)

stats_per_sp.sort_values('Service Point')

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance
0               1          0.02          4.78       1.468336            0.660
1               2          0.02          1.23       0.535486            0.520
2               3          0.04          1.18       0.493258            0.480
3               4          0.03          2.34       0.655558            0.640
4               5          0.03          3.94       1.102515            1.030
5               6          0.16          1.21       0.706185            0.710
6               7          0.03         22.43       4.910602            3.040
7               8          0.06          9.25       1.003021            0.960
8               9          0.23          1.34       0.642720            0.690
9              10          0.07          2.06       0.853276            0.840
10             11          0.05         14.55       4.932482            6.100
11             12          0.04          3.97       1.431926            1.270
12             13          0.01          1.57       0.665975            0.740
13             14          0.02          7.78       1.548702            1.140
14             15          0.01          1.14       0.507834            0.520
15             16          0.35         35.03      10.793079            5.730
16             17          0.01          1.87       0.568618            0.570
17             18          0.09          2.93       1.235521            1.280
18             19          0.01          1.23       0.539394            0.560
19             20          0.02          2.69       0.462565            0.510
20             21          0.01         16.30       6.395153            5.470
21             22          0.04          3.23       0.760828            0.750
22             23          0.21          9.75       2.102192            1.730
23             24          0.02         20.74       2.680263            0.710
24             25          0.00          9.96       1.202830            1.150
25             26          0.01         22.11       5.817752            6.230
26             27          0.11         18.20       0.876183            0.180
27             28          0.14          2.09       0.569558            0.340
28             29          0.05          1.18       0.645480            0.670
29             30          0.02          3.26       0.841446            0.680
30             31          0.11         23.65       6.878007            8.580
31             32          0.03          3.56       0.640072            0.570
32             33          0.06          9.31       5.560522            4.610
33             34          0.03          1.82       0.598159            0.430
34             35          0.17         21.67       5.490067            2.080
35             36          0.02          1.91       0.602731            0.580
36             37          0.01         11.69       1.226610            0.930
37             38          0.05         11.67       5.281492            7.200
38             39          0.01          1.46       0.513446            0.560
39             40          0.09          8.77       0.760383            0.850
40             41          0.02          1.42       0.629923            0.630
41             42          0.02          2.30       0.709692            0.580
42             43          0.12          1.25       0.481033            0.430
43             44          0.00          1.99       0.789896            0.680
44             45          0.04          0.96       0.587912            0.670
45             46          0.09         21.56       4.210525            0.400
46             47          0.06         19.28       7.201584            6.620
47             48          0.04         16.14       1.202011            1.095
48             49          0.03         22.95       4.269198            1.220
49             50          0.05          1.10       0.349209            0.180
50             51         

In [13]:
stats_per_sp['95th Percentile'] = cust_sp_df.groupby('CompA_SP')['MinDistTo_CompA_SP'].quantile(0.95)
stats_per_sp['5th Percentile'] = cust_sp_df.groupby('CompA_SP')['MinDistTo_CompA_SP'].quantile(0.05)
stats_per_sp.head(5)

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance  \
0              1          0.02          4.78       1.468336             0.66   
1              2          0.02          1.23       0.535486             0.52   
2              3          0.04          1.18       0.493258             0.48   
3              4          0.03          2.34       0.655558             0.64   
4              5          0.03          3.94       1.102515             1.03   

   95th Percentile  5th Percentile  
0              NaN             NaN  
1             4.51            0.25  
2             1.14            0.02  
3             0.99            0.12  
4             1.40            0.10

In [14]:
#Join statistics with the main service point information
join_df2 = pd.merge(join_df,stats_per_sp,on="Service Point",how="left")
join_df2.fillna(0, inplace=True)
join_df2.sort_values('Service Point', ascending=True).head(5)

Service Point                           Full Address   Latitude  Longitude  \
0              1  Ελευθερίου Βενιζέλου 1, Λευκωσία 2235  35.104199  33.379002   
1              2    Λεωφ. Ακροπόλεως 58Α, Λευκωσία 2012  35.149497  33.359752   
2              3       Λεωφ. Μακαρίου 39, Λευκωσία 1065  35.165882  33.362994   
3              4     Λεωφ. Στροβόλου 116, Λευκωσία 2042  35.143165  33.341816   
4              5       28ης Οκτωβρίου 8A, Λευκωσία 2414  35.164435  33.318236   

  District  NoOfCustomers  Min Distance  Max Distance  Mean Distance  \
0  Nicosia           1707          0.02          4.78       1.468336   
1  Nicosia           4061          0.02          1.23       0.535486   
2  Nicosia          23857          0.04          1.18       0.493258   
3  Nicosia           5558          0.03          2.34       0.655558   
4  Nicosia           4740          0.03          3.94       1.102515   

   Median Distance  95th Percentile  5th Percentile  
0             0.66             0.00            0.00  
1             0.52             4.51            0.25  
2             0.48             1.14            0.02  
3             0.64             0.99            0.12  
4             1.03             1.40            0.10

In [17]:
join_df2.info()
join_df2.to_csv('CompA_InitialStatsV4.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 56
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Service Point    57 non-null     int64  
 1   Full Address     57 non-null     object 
 2   Latitude         57 non-null     float64
 3   Longitude        57 non-null     float64
 4   District         57 non-null     object 
 5   NoOfCustomers    57 non-null     int64  
 6   Min Distance     57 non-null     float64
 7   Max Distance     57 non-null     float64
 8   Mean Distance    57 non-null     float64
 9   Median Distance  57 non-null     float64
 10  95th Percentile  57 non-null     float64
 11  5th Percentile   57 non-null     float64
dtypes: float64(8), int64(2), object(2)
memory usage: 5.8+ KB


In [15]:
#Repeat the process to get the metrics for Company B Existing Service Points
#dist_df = df[(df['lat'] <= lat1) & (df['lat'] >= lat2) & (df['lng'] <= lon1) & (df['lng'] >= lon2)]
dfB = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/distances_CompB.csv', delimiter=',', encoding='utf-8')
distB_df = dfB[(dfB['lat'] <= lat1) & (dfB['lat'] >= lat2) & (dfB['lng'] <= lon1) & (dfB['lng'] >= lon2)]
print(distB_df.head(20))
print(distB_df.info())

                                    ORGANISATION_NAME  REGISTRATION_NO  \
0                    THE SUN INSURANCE OFFICE LIMITED               11   
1                     COMPTOIR D' ESCOMPTE DE NICOSIE               13   
2                                    SIMMONS BROTHERS               32   
3                                  NEW STAR SHOE SHOP              118   
4                           CHARALAMBOS PIERI AND SON               17   
5   ΚΥΠΡΙΑΚΟΝ ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗ...               19   
6   ΕΜΠΟΡΙΚΟΝ ΚΑΙ ΒΙΟΜΗΧΑΝΙΚΟΝ ΕΠΙΜΕΛΗΤΗΡΙΟΝ ΛΕΥΚΩ...              502   
7   Ο ΚΥΠΡΙΑΚΟΣ ΣΥΝΔΕΣΜΟΣ ΙΔΙΟΚΤΗΤΩΝ ΑΚΙΝΗΤΩΝ ΛΙΜΙΤΕΔ             1293   
8                CYPRUS-AMERICAN BUSINESS ASSOCIATION            17284   
9                                               CYABA            17285   
10                                      AMCHAM CYPRUS            35841   
11                                   C.J. CACOYIANNIS               28   
12                                    

In [16]:
msrB_df = distB_df.select_dtypes(include=['number'])
msrB_df = msrB_df.drop(['lat_txt','lng_txt','lat','lng','address_seq_no','REGISTRATION_NO'], axis=1)
msrB_df['min_dist'] = msrB_df.min(axis='columns')
msrB_df['min_sp'] = msrB_df.idxmin(axis='columns')
msrB_df

Valid Seq      1      2      3      4      5      6      7      8  \
0          281510  17.54   2.17   5.36   2.26   7.27   3.50  14.64   0.68   
1          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
2          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
3          274852  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
4          262535  20.12   4.11   8.79   5.01  10.71   5.94  17.02   3.94   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
218733     267985  50.54  62.31  56.85  60.94  54.98  60.84  54.88  61.77   
218734     565102  22.75  37.92  34.37  33.68  35.19  38.52  46.17  36.20   
218735     439357  20.62  13.32  10.98  14.75   9.12  11.49   4.14  13.80   
218736     352550  21.75  39.87  35.21  35.71  35.45  40.04  45.63  38.20   
218737     195348  48.49  61.19  55.59  59.58  53.79  59.78  54.46  60.55   

            9  ...     35     36      37      38      39      40      41  \
0        2.14  ...  60.48  59.29   86.22   94.08   96.56   93.99   95.37   
1        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
2        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
3        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
4        3.19  ...  63.91  62.71   88.66   97.16   99.60   97.07   98.37   
...       ...  ...    ...    ...     ...     ...     ...     ...     ...   
218733  63.04  ...   4.17   5.51   65.24   50.91   54.43   50.91   54.89   
218734  34.17  ...  58.69  57.19  109.61  108.15  111.31  108.10  110.90   
218735  16.35  ...  50.50  49.51   72.09   79.76   82.19   79.67   80.96   
218736  36.73  ...  49.68  48.18  104.15  100.62  103.88  100.58  103.61   
218737  61.67  ...   0.51   1.99   68.70   54.73   58.25   54.73   58.71   

            42  min_dist  min_sp  
0        94.27      0.51      10  
1        97.29      0.51      13  
2        97.29      0.51      13  
3        97.29      0.51      13  
4        97.29      0.51      13  
...        ...       ...     ...  
218733   53.11      1.82      30  
218734  109.39      0.98      25  
218735   79.88      4.14       7  
218736  102.04      2.53      20  
218737   56.93      0.00      27  

[217788 rows x 45 columns]

In [18]:
#Keep only the the interesting columns
serv_ptsB_df = pd.DataFrame(msrB_df, columns=['min_dist', 'min_sp'])
serv_ptsB_df['min_sp'] = serv_ptsB_df['min_sp'].astype('int64')
serv_ptsB_df

min_dist  min_sp
0           0.51      10
1           0.51      13
2           0.51      13
3           0.51      13
4           0.51      13
...          ...     ...
218733      1.82      30
218734      0.98      25
218735      4.14       7
218736      2.53      20
218737      0.00      27

[217788 rows x 2 columns]

In [19]:
#Concatenate dataframes to get the closest service point per customer
custB_df = pd.DataFrame(distB_df, columns=['ORGANISATION_NAME','address_seq_no', 'FULL ADDRESS', 'lat', 'lng'])
custB_sp_df = pd.concat([custB_df,serv_ptsB_df],axis=1)
custB_sp_df.rename(columns={'address_seq_no':'AddressID'}, inplace=True)
custB_sp_df.rename(columns={'min_dist':'MinDistTo_CompB_SP'}, inplace=True)
custB_sp_df.rename(columns={'min_sp':'CompB_SP'}, inplace=True)
custB_sp_df

ORGANISATION_NAME  AddressID  \
0       THE SUN INSURANCE OFFICE LIMITED     281510   
1        COMPTOIR D' ESCOMPTE DE NICOSIE     274852   
2                       SIMMONS BROTHERS     274852   
3                     NEW STAR SHOE SHOP     274852   
4              CHARALAMBOS PIERI AND SON     262535   
...                                  ...        ...   
218733   BEAUTY STUDIO BY THEKLA LIMITED     267985   
218734                      ZI. RA 1 LTD     565102   
218735                 ACLA AUDITORS LTD     439357   
218736         ΣΥΛΒΕΣΤΡΟΥ ΚΑΙ ΠΗΛΑΔΗ ΛΤΔ     352550   
218737                       ILLIDAN LTD     195348   

                                             FULL ADDRESS        lat  \
0                    ΠΡΙΓΚΙΠΙΣΣΗΣ ΝΤΕ ΤΥΡΑΣ 12  ΛΕΥΚΩΣΙΑ   35.160221   
1                                               ΛΕΥΚΩΣΙΑ   35.185566   
2                                               ΛΕΥΚΩΣΙΑ   35.185566   
3                                               ΛΕΥΚΩΣΙΑ   35.185566   
4                               ARISTIDES ROAD  ΛΕΥΚΩΣΙΑ   35.185566   
...                                                   ...        ...   
218733  ΠΑΝΑΓΙΩΤΗ ΑΝΑΓΝΩΣΤΟΠΟΥΛΟΥ 68  ΚΑΤΩ ΠΟΛΕΜΙΔΙΑ 4...  34.685159   
218734                        ΓΛΑΔΣΤΩΝΟΣ 1  6023 ΛΑΡΝΑΚΑ   34.918222   
218735   ΠΕΝΤΑΔΑΚΤΥΛΟΥ 21  ΑΓΙΟΙ ΤΡΙΜΙΘΙΑΣ 2671 ΛΕΥΚΩΣΙΑ   35.114799   
218736              ΟΔΥΣΣΕΩΣ 10  ΤΕΡΣΕΦΑΝΟΥ 7562 ΛΑΡΝΑΚΑ   34.854814   
218737                     ΑΡΓΥΡΗ ΓΑΒΑΛΑ 3  3112 ΛΕΜΕΣΟΣ   34.678632   

              lng  MinDistTo_CompB_SP  CompB_SP  
0       33.360473                0.51        10  
1       33.382276                0.51        13  
2       33.382276                0.51        13  
3       33.382276                0.51        13  
4       33.382276                0.51        13  
...           ...                 ...       ...  
218733  33.000279                1.82        30  
218734  33.620063                0.98        25  
218735  33.211395                4.14         7  
218736  33.546889                2.53        20  
218737  33.041305                0.00        27  

[217788 rows x 7 columns]

In [66]:
custB_sp_df.to_csv('CompB_MinSP_AllData.csv', encoding='utf-8')

In [20]:
CompB_Loc = pd.read_csv('https://storage.googleapis.com/geolocation_data_project_cm/CompB_Locs.csv', delimiter=';', encoding='utf-8')
CompB_Loc.dropna(inplace=True)
CompB_Loc['AA'] = CompB_Loc['AA'].astype('int64')
CompB_Loc.rename(columns={'AA': 'Service Point'}, inplace=True)
CompB_Loc.head(5)

Service Point                                       Full Address  \
0              1     1, Synergatismou Str., Pera Chorio Nisou, 2572   
1              2  10-12, Grigori Afxentiou Ave., Agios Dometios,...   
2              3               124, Tseriou Avenue, Strovolos, 2044   
3              4               173, Athalassa Ave., Strovolos, 2025   
4              5      200, Arch. Makariou III Ave., Lakatamia, 2311   

    Latitude  Longitude District  
0  35.004875  33.393852  Nicosia  
1  35.171138  33.340676  Nicosia  
2  35.118028  33.332058  Nicosia  
3  35.141553  33.370235  Nicosia  
4  35.108642  33.311404  Nicosia

In [21]:
#Some baseline counts regarding existing physical branch network of Company B
customers_per_spB = serv_ptsB_df.groupby("min_sp").count().reset_index()
customers_per_spB.columns = ['Service Point', 'NoOfCustomers']
customers_per_spB['Service Point'] = customers_per_spB['Service Point'].astype('int64')

join_dfB = pd.merge(customers_per_spB,CompB_Loc,on="Service Point",how="inner")
join_dfB.sort_values('NoOfCustomers', ascending=False)
join_dfB.head(5)

Service Point  NoOfCustomers  \
0              1           2885   
1              2           4242   
2              3           3207   
3              4           2912   
4              5           3047   

                                        Full Address   Latitude  Longitude  \
0     1, Synergatismou Str., Pera Chorio Nisou, 2572  35.004875  33.393852   
1  10-12, Grigori Afxentiou Ave., Agios Dometios,...  35.171138  33.340676   
2               124, Tseriou Avenue, Strovolos, 2044  35.118028  33.332058   
3               173, Athalassa Ave., Strovolos, 2025  35.141553  33.370235   
4      200, Arch. Makariou III Ave., Lakatamia, 2311  35.108642  33.311404   

  District  
0  Nicosia  
1  Nicosia  
2  Nicosia  
3  Nicosia  
4  Nicosia

In [23]:
#Statistics per service point
stats_per_spB = serv_ptsB_df.groupby(['min_sp']).aggregate([np.min,np.max,np.mean,
                                                            np.median]).reset_index()

stats_per_spB.columns = stats_per_spB.columns.map('_'.join)
stats_per_spB.rename(columns={'min_sp_':'Service Point'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_amin':'Min Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_amax':'Max Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_mean':'Mean Distance'}, inplace=True)
stats_per_spB.rename(columns={'min_dist_median':'Median Distance'}, inplace=True)

stats_per_spB.sort_values('Mean Distance')
stats_per_spB

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance
0               1          0.00         13.34       4.298163            2.830
1               2          0.00          3.03       0.679144            0.620
2               3          0.00          3.29       1.371204            1.300
3               4          0.00          1.60       0.561868            0.530
4               5          0.00          5.01       1.896242            1.640
5               6          0.03          4.30       1.222766            1.170
6               7          0.14         33.08       6.215708            4.140
7               8          0.01          2.33       0.801802            0.850
8               9          0.00         20.78       1.119660            0.890
9              10          0.03          1.35       0.631859            0.560
10             11          0.02          0.96       0.515932            0.560
11             12          0.40         35.10      14.073799           10.240
12             13          0.05         14.40       0.677474            0.510
13             14          0.01          2.48       1.306013            1.420
14             15          0.19         11.38       1.962708            2.230
15             16          0.01          2.46       0.863439            0.980
16             17          0.10         14.16       4.648678            1.390
17             18          0.00         25.23       3.673539            2.020
18             19          0.12         14.07       4.801266            1.300
19             20          0.00         12.05       5.140805            7.230
20             21          0.00          5.08       0.542881            0.270
21             22          0.00          3.22       0.787174            0.820
22             23          0.10          1.69       0.704618            0.630
23             24          0.10         23.96       8.058842            6.890
24             25          0.11          1.72       0.899118            0.980
25             26          0.00          3.16       0.626176            0.500
26             27          0.00          1.45       0.281485            0.070
27             28          0.00         18.27       7.660713            8.060
28             29          0.00          2.06       0.663794            0.690
29             30          0.00          2.68       1.024704            0.910
30             31          0.00         35.24      10.467394            3.490
31             32          0.00          3.34       1.753756            1.810
32             33          0.06         24.98       5.108962            6.720
33             34          0.04         18.26       1.967748            1.120
34             35          0.01          0.97       0.442479            0.490
35             36          0.00          1.35       0.531127            0.620
36             37          0.00         34.79       5.110650            0.280
37             38          0.00         21.97       3.690024            1.255
38             39          0.00          2.32       0.994085            0.940
39             40          0.05         24.46       2.867728            0.860
40             41          0.05         13.35       3.897254            2.000
41             42          0.02         21.90       1.642599            1.130

In [24]:
stats_per_spB['95th Percentile'] = custB_sp_df.groupby('CompB_SP')['MinDistTo_CompB_SP'].quantile(0.95)
stats_per_spB['5th Percentile'] = custB_sp_df.groupby('CompB_SP')['MinDistTo_CompB_SP'].quantile(0.05)
stats_per_spB

Service Point  Min Distance  Max Distance  Mean Distance  Median Distance  \
0               1          0.00         13.34       4.298163            2.830   
1               2          0.00          3.03       0.679144            0.620   
2               3          0.00          3.29       1.371204            1.300   
3               4          0.00          1.60       0.561868            0.530   
4               5          0.00          5.01       1.896242            1.640   
5               6          0.03          4.30       1.222766            1.170   
6               7          0.14         33.08       6.215708            4.140   
7               8          0.01          2.33       0.801802            0.850   
8               9          0.00         20.78       1.119660            0.890   
9              10          0.03          1.35       0.631859            0.560   
10             11          0.02          0.96       0.515932            0.560   
11             12          0.40         35.10      14.073799           10.240   
12             13          0.05         14.40       0.677474            0.510   
13             14          0.01          2.48       1.306013            1.420   
14             15          0.19         11.38       1.962708            2.230   
15             16          0.01          2.46       0.863439            0.980   
16             17          0.10         14.16       4.648678            1.390   
17             18          0.00         25.23       3.673539            2.020   
18             19          0.12         14.07       4.801266            1.300   
19             20          0.00         12.05       5.140805            7.230   
20             21          0.00          5.08       0.542881            0.270   
21             22          0.00          3.22       0.787174            0.820   
22             23          0.10          1.69       0.704618            0.630   
23             24          0.10         23.96       8.058842            6.890   
24             25          0.11          1.72       0.899118            0.980   
25             26          0.00          3.16       0.626176            0.500   
26             27          0.00          1.45       0.281485            0.070   
27             28          0.00         18.27       7.660713            8.060   
28             29          0.00          2.06       0.663794            0.690   
29             30          0.00          2.68       1.024704            0.910   
30             31          0.00         35.24      10.467394            3.490   
31             32          0.00          3.34       1.753756            1.810   
32             33          0.06         24.98       5.108962            6.720   
33             34          0.04         18.26       1.967748            1.120   
34             35          0.01          0.97       0.442479            0.490   
35             36          0.00          1.35       0.531127            0.620   
36             37          0.00         34.79       5.110650            0.280   
37             38          0.00         21.97       3.690024            1.255   
38             39          0.00          2.32       0.994085            0.940   
39             40          0.05         24.46       2.867728            0.860   
40             41          0.05         13.35       3.897254            2.000   
41             42          0.02         21.90       1.642599            1.130   

    95th Percentile  5th Percentile  
0               NaN             NaN  
1             10.27            0.02  
2              1.07            0.24  
3              2.63            0.45  
4              0.90            0.12  
5              4.38            0.00  
6              2.64            0.29  
7             15.10            0.22  
8              1.25            0.22  
9              1.94            0.44  
10             1.23            0.31  
11             0.87            0.16  
12            29.17       

In [25]:
#Join statistics with the main service point information
join_df2B = pd.merge(join_dfB,stats_per_spB,on="Service Point",how="inner")
join_df2B.sort_values('NoOfCustomers', ascending=False).head(5)

Service Point  NoOfCustomers  \
7               8          22259   
15             16          19724   
26             27          12147   
34             35          10804   
21             22           9098   

                                     Full Address   Latitude  Longitude  \
7     34, Demostheni Severi Avenue, Nicosia, 1080  35.159420  33.353050   
15          92, Digeni Akrita Ave., Nicosia, 1061  35.167700  33.371050   
26  108, Fr.Roosevelt Str.&Avanas, Limassol, 3011  34.678632  33.041305   
34            97, Gladstonos Str., Limassol, 3032  34.681440  33.045650   
21         2, Zenonos Kitieus Str., Larnaca, 6023  34.917684  33.635243   

    District  Min Distance  Max Distance  Mean Distance  Median Distance  \
7    Nicosia          0.01          2.33       0.801802             0.85   
15   Nicosia          0.01          2.46       0.863439             0.98   
26  Limassol          0.00          1.45       0.281485             0.07   
34  Limassol          0.01          0.97       0.442479             0.49   
21   Larnaca          0.00          3.22       0.787174             0.82   

    95th Percentile  5th Percentile  
7             15.10            0.22  
15             4.00            0.42  
26             1.56            0.01  
34             5.91            0.50  
21             1.49            0.18

In [73]:
join_df2B.info()
join_df2B.to_csv('CompB_InitialStats.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Service Point    42 non-null     int64  
 1   NoOfCustomers    42 non-null     int64  
 2   Full Address     42 non-null     object 
 3   Latitude         42 non-null     float64
 4   Longitude        42 non-null     float64
 5   District         42 non-null     object 
 6   Min Distance     42 non-null     float64
 7   Max Distance     42 non-null     float64
 8   Mean Distance    42 non-null     float64
 9   Median Distance  42 non-null     float64
 10  95th Percentile  41 non-null     float64
 11  5th Percentile   41 non-null     float64
dtypes: float64(8), int64(2), object(2)
memory usage: 4.3+ KB
